# MCM/ICM 2025 - TOPSIS评估 + 平滑S曲线预测至2070年

**更新要求：**
1. 使用TOPSIS方法评估10个发射场，删除最低3个
2. 平滑S形曲线渐进逼近365次/年（而非截断）
3. 预测延长至2070年

In [ ]:
# 导入库 (Jupyter Notebook版本 - 移除了 matplotlib.use('Agg'))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import warnings
warnings.filterwarnings('ignore')

# Jupyter内联显示图形
%matplotlib inline

# 设置字体
rcParams['font.sans-serif'] = ['DejaVu Sans']
rcParams['axes.unicode_minus'] = False
rcParams['figure.figsize'] = [12, 8]
rcParams['figure.dpi'] = 100

print("库导入成功！")

In [ ]:
# 数据定义 - 2016-2025年各发射场年度发射次数
years = np.array([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])

spaceport_data = {
    'Alaska': {
        'name': 'Pacific Spaceport Complex',
        'latitude': 57.4,
        'launches': np.array([0, 0, 0, 0, 2, 2, 1, 1, 0, 0]),
        'country': 'USA',
        'note': 'Very high latitude, mostly suborbital tests, declining'
    },
    'California': {
        'name': 'Vandenberg SFB', 
        'latitude': 34.7,
        'launches': np.array([8, 12, 15, 10, 8, 14, 22, 25, 28, 66]),
        'country': 'USA',
        'note': 'Primary polar/SSO launch site, explosive growth'
    },
    'Texas': {
        'name': 'SpaceX Starbase',
        'latitude': 26.0,
        'launches': np.array([0, 0, 0, 0, 0, 0, 0, 3, 5, 5]),
        'country': 'USA',
        'note': 'New site, Starship testing, high potential'
    },
    'Florida': {
        'name': 'Cape Canaveral/KSC',
        'latitude': 28.5,
        'launches': np.array([18, 29, 31, 21, 31, 45, 57, 72, 85, 109]),
        'country': 'USA',
        'note': 'Primary US launch site, highest activity globally'
    },
    'Virginia': {
        'name': 'Wallops/MARS',
        'latitude': 37.8,
        'launches': np.array([1, 2, 3, 3, 4, 4, 4, 7, 8, 1]),
        'country': 'USA',
        'note': 'Antares retired, Rocket Lab occasional, declining'
    },
    'Kazakhstan': {
        'name': 'Baikonur Cosmodrome',
        'latitude': 45.6,
        'launches': np.array([17, 17, 16, 15, 12, 12, 9, 9, 8, 6]),
        'country': 'Kazakhstan/Russia',
        'note': 'Declining due to Russia relocation to Vostochny'
    },
    'French Guiana': {
        'name': 'Guiana Space Centre',
        'latitude': 5.2,
        'launches': np.array([11, 11, 11, 9, 5, 6, 5, 4, 3, 7]),
        'country': 'France/ESA',
        'note': 'Best latitude, Ariane 6 transition'
    },
    'India': {
        'name': 'Satish Dhawan Space Centre',
        'latitude': 13.7,
        'launches': np.array([7, 5, 7, 6, 2, 2, 5, 7, 8, 5]),
        'country': 'India',
        'note': 'ISRO primary site, steady but limited growth'
    },
    'China-Taiyuan': {
        'name': 'Taiyuan Satellite Launch Center',
        'latitude': 37.5,
        'launches': np.array([8, 7, 11, 13, 10, 14, 12, 14, 15, 12]),
        'country': 'China',
        'note': 'Polar orbit launches, consistent'
    },
    'New Zealand': {
        'name': 'Mahia Peninsula (Rocket Lab)',
        'latitude': -39.3,
        'launches': np.array([0, 1, 3, 6, 6, 6, 9, 10, 16, 17]),
        'country': 'New Zealand',
        'note': 'Rocket Lab Electron, strong growth'
    }
}

print(f"已加载 {len(spaceport_data)} 个发射场数据")

In [ ]:
# TOPSIS 评估方法
def topsis_scorer(df, weights, negative_indicators=None):
    """TOPSIS评估方法"""
    data = df.copy().astype(float)
    weights = np.array(weights)
    negative_indicators = negative_indicators if negative_indicators else []
    
    # 1. 向量归一化
    norm_data = data / np.sqrt((data**2).sum(axis=0))
    
    # 2. 加权归一化矩阵
    weighted_data = norm_data * weights
    
    # 3. 确定理想解和负理想解
    ideal_best = []
    ideal_worst = []
    
    for col in data.columns:
        if col in negative_indicators:
            ideal_best.append(weighted_data[col].min())
            ideal_worst.append(weighted_data[col].max())
        else:
            ideal_best.append(weighted_data[col].max())
            ideal_worst.append(weighted_data[col].min())
    
    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)
    
    # 4. 计算欧氏距离
    d_pos = np.sqrt(((weighted_data - ideal_best)**2).sum(axis=1))
    d_neg = np.sqrt(((weighted_data - ideal_worst)**2).sum(axis=1))
    
    # 5. 计算相对贴近度 (TOPSIS得分)
    scores = d_neg / (d_pos + d_neg)
    
    # 整理结果
    results = df.copy()
    results['Distance_Pos'] = d_pos
    results['Distance_Neg'] = d_neg
    results['TOPSIS_Score'] = scores
    results['Rank'] = scores.rank(ascending=False).astype(int)
    
    return results.sort_values('Rank')


def calculate_topsis_metrics(data, years):
    """计算TOPSIS所需的评估指标"""
    metrics_list = []
    
    for site, info in data.items():
        launches = info['launches']
        
        total_10y = np.sum(launches)
        latest_2025 = launches[-1]
        avg_5y = np.mean(launches[-5:])
        
        nonzero_idx = np.where(launches > 0)[0]
        if len(nonzero_idx) >= 2:
            start_idx = nonzero_idx[0]
            start_val = max(launches[start_idx], 0.5)
            end_val = max(launches[-1], 0.1)
            years_growth = len(launches) - start_idx - 1
            if years_growth > 0:
                cagr = (end_val / start_val) ** (1/years_growth) - 1
            else:
                cagr = 0
        else:
            cagr = -0.3 if total_10y < 5 else 0
        
        recent = launches[-3:]
        if np.sum(recent) > 0:
            slope = np.polyfit(np.arange(3), recent, 1)[0]
        else:
            slope = -1
        
        lat_abs = abs(info['latitude'])
        
        metrics_list.append({
            'Site': site,
            'Total_10Y': total_10y,
            'Launches_2025': latest_2025,
            'Avg_5Y': avg_5y,
            'CAGR': max(cagr, -0.5),
            'Trend_Slope': max(slope, -5),
            'Latitude_Abs': lat_abs
        })
    
    return pd.DataFrame(metrics_list).set_index('Site')


def evaluate_with_topsis(data, years, n_close=3):
    """使用TOPSIS方法评估发射场"""
    metrics_df = calculate_topsis_metrics(data, years)
    weights = [0.20, 0.15, 0.15, 0.20, 0.15, 0.15]
    negative_indicators = ['Latitude_Abs']
    results = topsis_scorer(metrics_df, weights, negative_indicators)
    
    all_ranked = results.sort_values('TOPSIS_Score', ascending=True)
    close_sites = list(all_ranked.index[:n_close])
    keep_sites = list(all_ranked.index[n_close:])
    
    return results, keep_sites, close_sites, metrics_df

print("TOPSIS函数定义完成！")

In [ ]:
# 预测函数
def gm11_predict(x0, n_predict):
    """标准GM(1,1)灰色预测"""
    n = len(x0)
    x0 = np.array(x0, dtype=float)
    x0 = np.maximum(x0, 0.1)
    
    x1 = np.cumsum(x0)
    z1 = 0.5 * (x1[:-1] + x1[1:])
    
    B = np.column_stack([-z1, np.ones(n-1)])
    Y = x0[1:].reshape(-1, 1)
    
    try:
        params = np.linalg.lstsq(B, Y, rcond=None)[0]
        a, b = params[0, 0], params[1, 0]
    except:
        return np.full(n + n_predict, x0[-1])
    
    predictions = []
    for k in range(n + n_predict):
        x1_pred = (x0[0] - b/a) * np.exp(-a * k) + b/a
        predictions.append(x1_pred)
    
    predictions = np.array(predictions)
    x0_pred = np.diff(predictions, prepend=0)
    x0_pred[0] = x0[0]
    return np.maximum(x0_pred, 0)


def smooth_asymptotic_prediction_high_growth(x0, years_future, K=365, target_year=45):
    """高增长站点的平滑S曲线预测"""
    N0 = max(x0[-1], 1)
    ratio = (K - N0) / N0
    r = -np.log(0.02 / ratio) / target_year if ratio > 0 else 0.1
    r = np.clip(r, 0.05, 0.25)
    
    pred = []
    for t in range(len(x0) + years_future):
        if t < len(x0):
            pred.append(x0[t])
        else:
            t_future = t - len(x0) + 1
            denominator = 1 + ratio * np.exp(-r * t_future)
            N_t = K / denominator
            pred.append(min(N_t, K * 0.995))
    
    return np.array(pred)


def smooth_asymptotic_prediction_stable(x0, years_future, K=365, growth_type='stable'):
    """稳定/衰退站点的预测"""
    gm_pred = gm11_predict(x0, years_future)
    recent_trend = x0[-3:].mean() - x0[:3].mean()
    
    if recent_trend < -2:
        pred = []
        for t in range(len(x0) + years_future):
            if t < len(x0):
                pred.append(x0[t])
            else:
                t_future = t - len(x0) + 1
                decay_rate = 0.03
                N_t = max(x0[-1] * np.exp(-decay_rate * t_future), max(1, x0[-1] * 0.2))
                pred.append(N_t)
        return np.array(pred)
    else:
        pred = []
        base = x0[-3:].mean()
        for t in range(len(x0) + years_future):
            if t < len(x0):
                pred.append(x0[t])
            else:
                t_future = t - len(x0) + 1
                growth_rate = 0.02
                N_t = base * (1 + growth_rate * t_future)
                cap = min(base * 5, 100)
                pred.append(min(N_t, cap))
        return np.array(pred)

print("预测函数定义完成！")

In [ ]:
# 执行TOPSIS评估
print("="*80)
print("MCM/ICM 2025 - TOPSIS评估 + 平滑S曲线预测至2070年")
print("="*80)

topsis_results, keep_sites, close_sites, metrics_df = evaluate_with_topsis(
    spaceport_data, years, n_close=3
)

print("\n指标权重分配:")
print("  - 历史发射总量 (Total_10Y): 20%  [效益型]")
print("  - 2025年发射量 (Launches_2025): 15%  [效益型]")
print("  - 近5年平均 (Avg_5Y): 15%  [效益型]")
print("  - 年复合增长率 (CAGR): 20%  [效益型]")
print("  - 趋势斜率 (Trend_Slope): 15%  [效益型]")
print("  - 纬度绝对值 (Latitude_Abs): 15%  [成本型]")

print("\n" + "-"*80)
print("TOPSIS评估结果")
print("-"*80)
display(topsis_results[['Total_10Y', 'Launches_2025', 'CAGR', 'Latitude_Abs', 'TOPSIS_Score', 'Rank']])

print(f"\n【保留】({len(keep_sites)}个): {', '.join(keep_sites)}")
print(f"【关闭】({len(close_sites)}个): {', '.join(close_sites)}")

In [ ]:
# 执行预测
years_predict = np.arange(2016, 2071)
n_history = len(years)
n_future = len(years_predict) - n_history
K = 365

high_growth_sites = ['Florida', 'California', 'Texas', 'New Zealand']
stable_sites = [s for s in keep_sites if s not in high_growth_sites]

predictions = {}
for site in keep_sites:
    x0 = spaceport_data[site]['launches']
    
    if site in high_growth_sites:
        pred = smooth_asymptotic_prediction_high_growth(x0, n_future, K=K, target_year=45)
    else:
        pred = smooth_asymptotic_prediction_stable(x0, n_future, K=K)
    
    predictions[site] = pred

# 打印预测表
print("\n平滑S曲线预测 (2025-2070)")
print("="*72)
print(f"{'Site':<18} {'2025':>7} {'2030':>7} {'2040':>7} {'2050':>7} {'2060':>7} {'2070':>7}")
print("-"*72)

total_by_year = {}
for year in [2025, 2030, 2040, 2050, 2060, 2070]:
    idx = np.where(years_predict == year)[0][0]
    total_by_year[year] = sum(predictions[s][idx] for s in keep_sites)

for site in sorted(keep_sites, key=lambda s: predictions[s][-1], reverse=True):
    pred = predictions[site]
    vals = []
    for year in [2025, 2030, 2040, 2050, 2060, 2070]:
        idx = np.where(years_predict == year)[0][0]
        vals.append(pred[idx])
    growth_type = "HIGH" if site in high_growth_sites else "STABLE"
    print(f"{site:<18} {vals[0]:>7.1f} {vals[1]:>7.1f} {vals[2]:>7.1f} {vals[3]:>7.1f} {vals[4]:>7.1f} {vals[5]:>7.1f}  [{growth_type}]")

print("-"*72)
print(f"{'TOTAL':<18} {total_by_year[2025]:>7.0f} {total_by_year[2030]:>7.0f} {total_by_year[2040]:>7.0f} "
      f"{total_by_year[2050]:>7.0f} {total_by_year[2060]:>7.0f} {total_by_year[2070]:>7.0f}")

In [ ]:
# 图1: 预测曲线
all_sites = list(spaceport_data.keys())
colors_all = plt.cm.tab10(np.linspace(0, 1, 10))
color_map = {site: colors_all[i] for i, site in enumerate(all_sites)}

fig1, ax1 = plt.subplots(figsize=(14, 8))

for site in keep_sites:
    launches = spaceport_data[site]['launches']
    pred = predictions[site]
    ax1.plot(years, launches, 'o-', color=color_map[site], 
            label=f'{site}', markersize=5, linewidth=2)
    ax1.plot(years_predict[n_history:], pred[n_history:], '--', 
            color=color_map[site], alpha=0.7, linewidth=2)

for site in close_sites:
    launches = spaceport_data[site]['launches']
    ax1.plot(years, launches, 'x--', color='gray', alpha=0.4, 
            label=f'{site} (CLOSED)', markersize=4, linewidth=1)

ax1.axvline(x=2025.5, color='red', linestyle=':', alpha=0.7, label='Prediction Start')
ax1.axhline(y=K, color='darkred', linestyle='--', alpha=0.5, label=f'K={K}/year')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Annual Launches', fontsize=12)
ax1.set_title('Spaceport Launch Predictions (2016-2070)', fontsize=14)
ax1.legend(loc='upper left', fontsize=8, ncol=2)
ax1.set_xlim(2016, 2070)
ax1.set_ylim(0, K + 30)
ax1.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/spaceport_prediction_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 图2: 2070年排名
fig2, ax2 = plt.subplots(figsize=(12, 8))

idx_2070 = np.where(years_predict == 2070)[0][0]
pred_2070 = {site: predictions[site][idx_2070] for site in keep_sites}
sorted_2070 = sorted(pred_2070.items(), key=lambda x: x[1], reverse=True)
sites_sorted = [x[0] for x in sorted_2070]
values_sorted = [x[1] for x in sorted_2070]
colors_sorted = [color_map[s] for s in sites_sorted]

bars = ax2.barh(range(len(sites_sorted)), values_sorted, color=colors_sorted, alpha=0.8, edgecolor='black')
ax2.set_yticks(range(len(sites_sorted)))
ax2.set_yticklabels([f'{s}\n(lat={spaceport_data[s]["latitude"]:.1f}°)' for s in sites_sorted], fontsize=9)
ax2.axvline(x=K, color='darkred', linestyle='--', alpha=0.7, label=f'K={K}')
ax2.set_xlabel('Predicted Launches in 2070', fontsize=12)
ax2.set_title(f'2070 Launch Capacity Ranking ({len(keep_sites)} Active Sites)', fontsize=14)
for bar, val in zip(bars, values_sorted):
    pct = val / K * 100
    ax2.text(val + 5, bar.get_y() + bar.get_height()/2, 
            f'{val:.0f} ({pct:.0f}%)', va='center', fontsize=10, fontweight='bold')
ax2.set_xlim(0, K + 80)
ax2.invert_yaxis()
ax2.legend()
plt.tight_layout()
plt.savefig('../results/spaceport_2070_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 图3: TOPSIS得分
fig3, ax3 = plt.subplots(figsize=(12, 7))

all_sites_sorted = list(topsis_results.index)
x_pos = np.arange(len(all_sites_sorted))
score_vals = [topsis_results.loc[s, 'TOPSIS_Score'] for s in all_sites_sorted]
colors_score = ['forestgreen' if s in keep_sites else 'crimson' for s in all_sites_sorted]

bars3 = ax3.bar(x_pos, score_vals, color=colors_score, alpha=0.8, edgecolor='black')

threshold = (topsis_results.iloc[2]['TOPSIS_Score'] + topsis_results.iloc[3]['TOPSIS_Score']) / 2
ax3.axhline(y=threshold, color='orange', linestyle='--', linewidth=2, 
           alpha=0.8, label=f'Closure Threshold≈{threshold:.3f}')

ax3.set_xticks(x_pos)
ax3.set_xticklabels(all_sites_sorted, rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('TOPSIS Score', fontsize=12)
ax3.set_title('TOPSIS Evaluation Scores (Green=KEEP, Red=CLOSE)', fontsize=14)
ax3.legend(loc='upper right')
ax3.grid(True, axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars3, score_vals)):
    ax3.text(bar.get_x() + bar.get_width()/2, val + 0.01, 
            f'{val:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('../results/spaceport_topsis_scores.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 图4: TOPSIS气泡图
fig4, ax4 = plt.subplots(figsize=(16, 11))

all_scores = [topsis_results.loc[s, 'TOPSIS_Score'] for s in all_sites]
score_min, score_max = min(all_scores), max(all_scores)

cmap = plt.cm.RdYlGn
norm = Normalize(vmin=score_min, vmax=score_max)

annotation_offsets = {
    'Florida': (15, -25), 'California': (15, 10), 'New Zealand': (15, 10),
    'Texas': (-80, 15), 'China-Taiyuan': (15, 10), 'French Guiana': (15, -20),
    'India': (-70, 15), 'Virginia': (15, 10), 'Kazakhstan': (-85, -10), 'Alaska': (15, 10),
}

for site in all_sites:
    lat = spaceport_data[site]['latitude']
    launches_2025 = spaceport_data[site]['launches'][-1]
    topsis_score = topsis_results.loc[site, 'TOPSIS_Score']
    
    score_normalized = (topsis_score - score_min) / (score_max - score_min)
    size = 200 + score_normalized ** 1.5 * 2300
    bubble_color = cmap(norm(topsis_score))
    
    if site in keep_sites:
        marker, edge, alpha, linewidth = 'o', 'black', 0.85, 2
    else:
        marker, edge, alpha, linewidth = 'X', 'darkred', 0.65, 2.5
    
    ax4.scatter(lat, launches_2025, s=size, c=[bubble_color], alpha=alpha, 
               marker=marker, edgecolors=edge, linewidths=linewidth)
    
    offset = annotation_offsets.get(site, (15, 8))
    fontweight = 'bold' if site in keep_sites else 'normal'
    ax4.annotate(f'{site}\n(S={topsis_score:.3f})', (lat, launches_2025), 
                fontsize=9, alpha=0.95, xytext=offset, textcoords='offset points',
                fontweight=fontweight,
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))

ax4.axvline(x=0, color='blue', linestyle='--', alpha=0.7, linewidth=2, label='Equator')
ax4.set_xlabel('Latitude (degrees)', fontsize=13)
ax4.set_ylabel('Launches in 2025', fontsize=13)
ax4.set_title('TOPSIS Analysis: Latitude vs Launch Activity', fontsize=15)
ax4.set_xlim(-55, 75)
ax4.set_ylim(-15, 140)
ax4.grid(True, alpha=0.3)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax4, shrink=0.6, aspect=20, pad=0.02)
cbar.set_label('TOPSIS Score', fontsize=11)
ax4.legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.savefig('../results/spaceport_topsis_bubble.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n所有图像已保存到 results/ 目录！")